# Capsule Network for RL

In [1]:
import tensorflow as tf
import cv2
import sys
import time 
sys.path.append("game/")
import wrapped_flappy_bird as game
import random
import numpy as np
from collections import deque
from matplotlib import pyplot as plt

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
epsilon = 1e-9
iter_routing = 2
train_freq = 10

In [3]:
GAME = 'bird' # the name of the game being played for log files
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 1000 # timesteps to observe before training
EXPLORE = 3000000 # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.1 # starting value of epsilon
REPLAY_MEMORY = 500000 # number of previous transitions to remember
BATCH = 32 # size of minibatch

# UPDATED CAPSULE NETWORK

In [4]:
def squash(vector):
    vec_squared_norm = reduce_sum(tf.square(vector), -2, keepdims=True)
    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)
    vec_squashed = scalar_factor * vector  # element-wise
    return(vec_squashed)
def routing(input, b_IJ):
    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    W = tf.get_variable('Weight', shape=(1, 1024, 160, 8, 1), dtype=tf.float32,
                        initializer=tf.random_normal_initializer(stddev=0.01))
    biases = tf.get_variable('bias', shape=(1, 1, 10, 16, 1))
    # A better solution is using element-wise multiply, reduce_sum and reshape
    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as
    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and
    # reshape to [a, c]
    input = tf.tile(input, [1, 1, 160, 1, 1])
    #assert input.get_shape() == [cfg.batch_size, 1024, 160, 8, 1]

    u_hat = reduce_sum(W * input, axis=3, keepdims=True)
    u_hat = tf.reshape(u_hat, shape=[-1, 1024, 10, 16, 1])
    #assert u_hat.get_shape() == [cfg.batch_size, 1024, 10, 16, 1]

    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(iter_routing):
        with tf.variable_scope('iter_' + str(r_iter)):
            # line 4:
            # => [batch_size, 1024, 10, 1, 1]
            c_IJ = softmax(b_IJ, axis=2)

            # At last iteration, use `u_hat` in order to receive gradients from the following graph
            if r_iter == iter_routing - 1:
                # line 5:
                # weighting u_hat with c_IJ, element-wise in the last two dims
                # => [batch_size, 1024, 10, 16, 1]
                s_J = tf.multiply(c_IJ, u_hat)
                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                #assert s_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]

                # line 6:
                # squash using Eq.1,
                v_J = squash(s_J)
                #assert v_J.get_shape() == [cfg.batch_size, 1, 10, 16, 1]
            elif r_iter < iter_routing - 1:  # Inner iterations, do not apply backpropagation
                s_J = tf.multiply(c_IJ, u_hat_stopped)
                s_J = reduce_sum(s_J, axis=1, keepdims=True) + biases
                v_J = squash(s_J)

                # line 7:
                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1024, 10, 16, 1]
                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the
                # batch_size dim, resulting in [1, 1024, 10, 1, 1]
                v_J_tiled = tf.tile(v_J, [1, 1024, 1, 1, 1])
                u_produce_v = reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keepdims=True)
                #assert u_produce_v.get_shape() == [cfg.batch_size, 1024, 10, 1, 1]

                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)
                b_IJ += u_produce_v
    return(v_J)
# For version compatibility
def reduce_sum(input_tensor, axis=None, keepdims=False):
    return tf.reduce_sum(input_tensor, axis=axis, keepdims=keepdims)
# For version compatibility
def softmax(logits, axis=None):
    return tf.nn.softmax(logits, axis=axis)

In [5]:
def createNetwork():
    # input layer
    s= tf.placeholder("float", [None, 84, 84, 4])
    coeff = tf.placeholder(tf.float32, shape=(None, 1024, 10, 1, 1))
    ####################### New Network COnfiguration #####################    
    w_initializer, b_initializer = tf.random_normal_initializer(0., 0.01), tf.constant_initializer(0.01)
    w1 = tf.get_variable('w1',[8, 8, 4, 64],initializer=w_initializer)
    b1 = tf.get_variable('b1',[64],initializer=b_initializer)
    # Convolution Layer
    # Conv1, [batch_size, 20, 20, 64]
    l1 = tf.nn.conv2d(s, w1, strides=[1, 4, 4, 1], padding="VALID")
    
    conv1 = tf.nn.relu(tf.nn.bias_add(l1, b1))
    
    conv1 = tf.reshape(conv1,[-1,20,20,64])
    
    capsules = tf.contrib.layers.conv2d(conv1, 16 * 8, kernel_size=6, stride=2, padding="VALID",
                    activation_fn = tf.nn.relu,
                    weights_initializer = tf.contrib.layers.xavier_initializer(uniform=False),
                    biases_initializer=tf.constant_initializer(0))
    
    capsules = tf.reshape(capsules, (-1, 1024, 8, 1)) #Reshape to(batch_szie, 1152, 8, 1)
    
    capsules = squash(capsules)
    
    input_caps2 = tf.reshape(capsules, shape=(-1, 1024, 1, capsules.shape[-2].value, 1))
    
    caps2 = routing(input_caps2, coeff)
    
    vector_j = tf.reshape(caps2, shape=(-1, 160))
    #print(vector_j)
    q_eval = tf.contrib.layers.fully_connected(vector_j, num_outputs=ACTIONS, activation_fn=None)

    #print(q_eval)
    readout = q_eval
    return s, coeff, readout

In [6]:
def trainNetwork(s, coeff, readout, sess):
    tick = time.time()
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices = 1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cost)

    # open up a game state to communicate with emulator
    game_state = game.GameState()
    
    # store the previous observations in replay memory
    D = deque()
    
    # get the first state by doing nothing and preprocess the image to 84x84x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    x_t, r_0, terminal = game_state.frame_step(do_nothing)
    x_t = cv2.cvtColor(cv2.resize(x_t[:,:-110,:], (84, 84)), cv2.COLOR_BGR2GRAY)
    ret, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    sess.run(tf.global_variables_initializer())
    # saving and loading networks
    # saver = tf.train.Saver()
    # 
    sess.run(tf.global_variables_initializer())
    b_IJ1 = np.zeros((1, 1024, 10, 1, 1)).astype(np.float32) # batch_size=1
    b_IJ2 = np.zeros((BATCH, 1024, 10, 1, 1)).astype(np.float32) # batch_size=BATCH
    epsilon = INITIAL_EPSILON
    t = 0
    pscore = 0
    episode = 0
    loss = 0
    tick = time.time()
    action_freq = np.zeros(ACTIONS)
    while True:
        # choose an action epsilon greedily
        # readout_t = readout.eval(feed_dict = {s : [s_t].reshape((1,80,80,4))})[0]
        
        readout_t = readout.eval(feed_dict = {s:s_t.reshape((1,84,84,4)), coeff:b_IJ1})
        
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if random.random() <= epsilon or t <= OBSERVE:
            action_index = random.randrange(ACTIONS)
            a_t[action_index] = 1
        else:
            action_index = np.argmax(readout_t)
            a_t[action_index] = 1

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        x_t1_colored, r_t, terminal = game_state.frame_step(a_t)
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored[:,:-110,:], (84, 84)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (84, 84, 1))
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)
        action_freq += a_t
        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()
        
        # only train if done observing
        if t > OBSERVE and t%train_freq==0:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s:s_j1_batch, coeff:b_IJ2 })
            #readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                # if terminal only equals reward
                if minibatch[i][4]:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})
            loss = cost.eval(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch,
                coeff: b_IJ2})

        # update the old values
        s_t = s_t1
        t += 1

        # save progress every 10000 iterations
        #if t % 10000 == 0:
        #    saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        if(r_t==1):
            pscore += 1
        if(terminal ):#and (pscore > 5)
            print("ts", t,"action_freq",action_freq,"/e", round(epsilon,3),"/pscore",pscore,"/loss",loss,"/ Q_MAX %e" % np.max(readout_t))
            pscore = 0
        if(terminal == 1):
            episode +=1
            action_freq = np.zeros(ACTIONS)
        if(pscore > 50):
            print("Game_Ends_in Time:",int(time.time() - tick))
            break;
        # write info to files
        '''
        if t % 10000 <= 100:
            a_file.write(",".join([str(x) for x in readout_t]) + '\n')
            h_file.write(",".join([str(x) for x in h_fc1.eval(feed_dict={s:[s_t]})[0]]) + '\n')
            cv2.imwrite("logs_tetris/frame" + str(t) + ".png", x_t1)
        '''

In [7]:
def playGame():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    s, coeff, readout = createNetwork()
    trainNetwork(s, coeff, readout, sess)

In [ ]:
def main():
    playGame()

if __name__ == "__main__":
    tick = time.time()
    main()
    print("Game_Ends_in Time:",int(time.time() - tick))
    print("____________ END HERE _____________")

ts 49 action_freq [24. 25.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 3.078833e-03
ts 99 action_freq [28. 22.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 5.409284e-03
ts 149 action_freq [21. 29.] /e 0.1 /pscore 0 /loss 0 / Q_MAX -1.006978e-02
ts 199 action_freq [27. 23.] /e 0.1 /pscore 0 /loss 0 / Q_MAX -3.862311e-04
ts 249 action_freq [25. 25.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 2.676354e-02
ts 299 action_freq [26. 24.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 3.166082e-03
ts 349 action_freq [27. 23.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 2.676354e-02
ts 399 action_freq [24. 26.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 5.762247e-03
ts 449 action_freq [26. 24.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 1.511778e-02
ts 499 action_freq [25. 25.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 1.438736e-02
ts 549 action_freq [20. 30.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 9.572075e-03
ts 599 action_freq [24. 26.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 7.844073e-03
ts 649 action_freq [23. 27.] /e 0.1 /pscore 0 /loss 0 / Q_MAX 6.105557e-03
ts 699 action_freq [31. 1

ts 4928 action_freq [24. 26.] /e 0.1 /pscore 0 /loss 0.014120961 / Q_MAX -8.102278e-01
ts 4978 action_freq [14. 36.] /e 0.1 /pscore 0 /loss 0.024618758 / Q_MAX -8.924992e-01
ts 5038 action_freq [43. 17.] /e 0.1 /pscore 0 /loss 0.019017983 / Q_MAX 3.485318e+00
ts 5088 action_freq [31. 19.] /e 0.1 /pscore 0 /loss 0.018482523 / Q_MAX -3.293853e-01
ts 5138 action_freq [15. 35.] /e 0.1 /pscore 0 /loss 0.010801971 / Q_MAX -1.504183e+00
ts 5188 action_freq [29. 21.] /e 0.1 /pscore 0 /loss 0.021926899 / Q_MAX -1.516215e+00
ts 5238 action_freq [31. 19.] /e 0.1 /pscore 0 /loss 0.009256147 / Q_MAX -1.343125e+00
ts 5296 action_freq [41. 17.] /e 0.1 /pscore 0 /loss 0.012469083 / Q_MAX 3.755968e+00
ts 5346 action_freq [20. 30.] /e 0.1 /pscore 0 /loss 0.028921323 / Q_MAX -3.937433e-01
ts 5396 action_freq [17. 33.] /e 0.1 /pscore 0 /loss 0.008026065 / Q_MAX -8.829466e-01
ts 5446 action_freq [37. 13.] /e 0.1 /pscore 0 /loss 0.013610445 / Q_MAX -7.387761e-01
ts 5496 action_freq [40. 10.] /e 0.1 /pscore 

ts 10561 action_freq [43.  7.] /e 0.1 /pscore 0 /loss 0.045064136 / Q_MAX -3.452866e-01
ts 10611 action_freq [43.  7.] /e 0.1 /pscore 0 /loss 0.20621456 / Q_MAX -8.050400e-01
ts 10734 action_freq [109.  14.] /e 0.1 /pscore 2 /loss 0.276666 / Q_MAX -1.335990e+00
ts 10784 action_freq [32. 18.] /e 0.1 /pscore 0 /loss 0.029483171 / Q_MAX -2.466986e+00
ts 10834 action_freq [40. 10.] /e 0.1 /pscore 0 /loss 0.08584501 / Q_MAX -9.593888e-01
ts 10884 action_freq [28. 22.] /e 0.1 /pscore 0 /loss 0.5191419 / Q_MAX -1.359167e+00
ts 10934 action_freq [42.  8.] /e 0.1 /pscore 0 /loss 0.047883153 / Q_MAX -6.036069e-01
ts 11020 action_freq [71. 15.] /e 0.1 /pscore 1 /loss 0.025402404 / Q_MAX 5.760378e-01
ts 11070 action_freq [34. 16.] /e 0.1 /pscore 0 /loss 0.07222192 / Q_MAX 7.550623e-02
ts 11194 action_freq [106.  18.] /e 0.1 /pscore 2 /loss 0.13397548 / Q_MAX 2.658289e+00
ts 11244 action_freq [39. 11.] /e 0.1 /pscore 0 /loss 0.06313094 / Q_MAX 7.028741e-01
ts 11370 action_freq [110.  16.] /e 0.1 /p

ts 18290 action_freq [39. 11.] /e 0.099 /pscore 0 /loss 0.11991741 / Q_MAX -1.038910e+00
ts 18340 action_freq [44.  6.] /e 0.099 /pscore 0 /loss 0.055699352 / Q_MAX -6.453981e-01
ts 18399 action_freq [51.  8.] /e 0.099 /pscore 0 /loss 0.19137338 / Q_MAX -6.795680e-01
ts 18449 action_freq [44.  6.] /e 0.099 /pscore 0 /loss 0.14094645 / Q_MAX 6.812573e-01
ts 18536 action_freq [71. 16.] /e 0.099 /pscore 1 /loss 0.27790856 / Q_MAX 5.831531e-02
ts 18628 action_freq [79. 13.] /e 0.099 /pscore 1 /loss 0.19171111 / Q_MAX -1.216308e+00
ts 18726 action_freq [88. 10.] /e 0.099 /pscore 2 /loss 0.058988225 / Q_MAX 2.344613e+00
ts 18815 action_freq [79. 10.] /e 0.099 /pscore 1 /loss 0.06456471 / Q_MAX 7.866440e-01
ts 18867 action_freq [43.  9.] /e 0.099 /pscore 0 /loss 0.04498833 / Q_MAX 2.582504e+00
ts 19028 action_freq [143.  18.] /e 0.099 /pscore 3 /loss 0.06537008 / Q_MAX 2.621558e+00
ts 19079 action_freq [43.  8.] /e 0.099 /pscore 0 /loss 0.040771015 / Q_MAX -5.035661e-01
ts 19129 action_freq [

ts 26964 action_freq [127.  14.] /e 0.099 /pscore 3 /loss 0.24857865 / Q_MAX 4.862977e+00
ts 27014 action_freq [43.  7.] /e 0.099 /pscore 0 /loss 0.12485136 / Q_MAX 1.326251e+00
ts 27101 action_freq [80.  7.] /e 0.099 /pscore 1 /loss 0.100683466 / Q_MAX 2.894945e+00
ts 27188 action_freq [81.  6.] /e 0.099 /pscore 1 /loss 0.060982704 / Q_MAX 3.982066e+00
ts 27348 action_freq [142.  18.] /e 0.099 /pscore 3 /loss 0.5865413 / Q_MAX 2.853079e-01
ts 27492 action_freq [132.  12.] /e 0.099 /pscore 3 /loss 0.12996276 / Q_MAX 3.893101e+00
ts 27542 action_freq [44.  6.] /e 0.099 /pscore 0 /loss 0.057705358 / Q_MAX -1.357237e+00
ts 27592 action_freq [42.  8.] /e 0.099 /pscore 0 /loss 0.26413748 / Q_MAX -2.532436e+00
ts 27727 action_freq [122.  13.] /e 0.099 /pscore 2 /loss 0.07635939 / Q_MAX -4.939063e-01
ts 27864 action_freq [125.  12.] /e 0.099 /pscore 3 /loss 0.05110716 / Q_MAX -3.477673e-01
ts 27950 action_freq [76. 10.] /e 0.099 /pscore 1 /loss 0.120582275 / Q_MAX 8.978046e-01
ts 28088 action

...

ts 442100 action_freq [60. 10.] /e 0.085 /pscore 1 /loss 0.11294907 / Q_MAX 7.672054e+00
ts 442150 action_freq [41.  9.] /e 0.085 /pscore 0 /loss 0.6420782 / Q_MAX -1.105171e+00
ts 442200 action_freq [36. 14.] /e 0.085 /pscore 0 /loss 0.050352827 / Q_MAX -9.777473e-01
ts 442250 action_freq [41.  9.] /e 0.085 /pscore 0 /loss 0.11044721 / Q_MAX -9.077282e-01
ts 442300 action_freq [37. 13.] /e 0.085 /pscore 0 /loss 0.055322543 / Q_MAX -6.979119e-01
ts 442350 action_freq [42.  8.] /e 0.085 /pscore 0 /loss 0.031830605 / Q_MAX -7.116721e-01
ts 442400 action_freq [36. 14.] /e 0.085 /pscore 0 /loss 0.08436702 / Q_MAX -7.979437e-01
ts 442460 action_freq [43. 17.] /e 0.085 /pscore 0 /loss 0.37309316 / Q_MAX -1.255427e+00
ts 442510 action_freq [26. 24.] /e 0.085 /pscore 0 /loss 0.18605383 / Q_MAX -6.690053e-01
ts 442560 action_freq [28. 22.] /e 0.085 /pscore 0 /loss 0.7876338 / Q_MAX -6.176435e-01
ts 442626 action_freq [47. 19.] /e 0.085 /pscore 1 /loss 0.07830317 / Q_MAX 5.698147e-01
ts 442712 a

ts 449583 action_freq [82. 22.] /e 0.085 /pscore 2 /loss 0.5322749 / Q_MAX 7.417735e+00
ts 449742 action_freq [133.  26.] /e 0.085 /pscore 3 /loss 0.21690026 / Q_MAX -1.035913e+00
ts 449792 action_freq [43.  7.] /e 0.085 /pscore 0 /loss 0.080841556 / Q_MAX -9.836123e-01
ts 449842 action_freq [22. 28.] /e 0.085 /pscore 0 /loss 0.05009351 / Q_MAX -1.167781e+00
ts 449975 action_freq [116.  17.] /e 0.085 /pscore 2 /loss 0.09046871 / Q_MAX -5.674681e-02
ts 450038 action_freq [48. 15.] /e 0.085 /pscore 1 /loss 0.05636124 / Q_MAX -1.795353e+00
ts 450204 action_freq [145.  21.] /e 0.085 /pscore 3 /loss 0.46739197 / Q_MAX -4.481440e-01
ts 450254 action_freq [33. 17.] /e 0.085 /pscore 0 /loss 0.08925899 / Q_MAX -8.543571e-01
ts 450304 action_freq [35. 15.] /e 0.085 /pscore 0 /loss 0.6158324 / Q_MAX -1.037276e+00
ts 450354 action_freq [39. 11.] /e 0.085 /pscore 0 /loss 0.16822532 / Q_MAX -6.204215e-01
ts 450440 action_freq [62. 24.] /e 0.085 /pscore 1 /loss 0.23086618 / Q_MAX -8.216481e-01
ts 450

ts 457986 action_freq [44. 13.] /e 0.085 /pscore 0 /loss 0.056180187 / Q_MAX -1.607243e+00
ts 458146 action_freq [123.  37.] /e 0.085 /pscore 3 /loss 0.3220033 / Q_MAX -1.841399e+00
ts 458232 action_freq [74. 12.] /e 0.085 /pscore 1 /loss 0.18627195 / Q_MAX -1.667225e+00
ts 458282 action_freq [40. 10.] /e 0.085 /pscore 0 /loss 0.11261902 / Q_MAX -1.580145e+00
ts 458406 action_freq [103.  21.] /e 0.085 /pscore 2 /loss 0.23848096 / Q_MAX -1.084284e+00
ts 458462 action_freq [37. 19.] /e 0.085 /pscore 0 /loss 0.34911606 / Q_MAX 8.303233e-01
ts 458600 action_freq [106.  32.] /e 0.085 /pscore 3 /loss 0.21179478 / Q_MAX -1.981890e+00
ts 458723 action_freq [105.  18.] /e 0.085 /pscore 2 /loss 0.04238324 / Q_MAX 3.150289e+00
ts 458848 action_freq [104.  21.] /e 0.085 /pscore 2 /loss 0.12745029 / Q_MAX -6.075767e-01
ts 458898 action_freq [36. 14.] /e 0.085 /pscore 0 /loss 0.045985337 / Q_MAX -6.120207e-01
ts 459065 action_freq [139.  28.] /e 0.085 /pscore 3 /loss 0.41621256 / Q_MAX -8.748946e-01

ts 465937 action_freq [100.  23.] /e 0.085 /pscore 2 /loss 0.02339465 / Q_MAX -1.151352e+00
ts 466029 action_freq [80. 12.] /e 0.085 /pscore 1 /loss 0.19312282 / Q_MAX -2.050653e+00
ts 466098 action_freq [53. 16.] /e 0.085 /pscore 1 /loss 0.6849136 / Q_MAX 5.198404e+00
ts 466370 action_freq [229.  43.] /e 0.085 /pscore 6 /loss 0.29936564 / Q_MAX -7.920926e-01
ts 466467 action_freq [82. 15.] /e 0.084 /pscore 1 /loss 0.18689504 / Q_MAX -2.610135e-01
ts 466518 action_freq [44.  7.] /e 0.084 /pscore 0 /loss 0.18641289 / Q_MAX 1.504015e+01
ts 466568 action_freq [42.  8.] /e 0.084 /pscore 0 /loss 0.4559425 / Q_MAX -7.810421e-01
ts 466618 action_freq [42.  8.] /e 0.084 /pscore 0 /loss 0.14425363 / Q_MAX -1.063175e+00
ts 466750 action_freq [116.  16.] /e 0.084 /pscore 2 /loss 0.05718545 / Q_MAX 2.280982e-01
ts 466948 action_freq [168.  30.] /e 0.084 /pscore 4 /loss 0.19969898 / Q_MAX -3.327788e-01
ts 467034 action_freq [69. 17.] /e 0.084 /pscore 1 /loss 0.103377536 / Q_MAX -8.387922e-01
ts 467

ts 473142 action_freq [59. 10.] /e 0.084 /pscore 1 /loss 0.16209868 / Q_MAX 1.220354e+01
ts 473271 action_freq [110.  19.] /e 0.084 /pscore 2 /loss 0.10082195 / Q_MAX 1.533747e+00
ts 473321 action_freq [34. 16.] /e 0.084 /pscore 0 /loss 0.38440922 / Q_MAX -3.451166e-01
ts 473373 action_freq [36. 16.] /e 0.084 /pscore 0 /loss 0.36642575 / Q_MAX 1.287201e+01
ts 473475 action_freq [80. 22.] /e 0.084 /pscore 2 /loss 0.11890688 / Q_MAX -5.687832e-01
ts 473525 action_freq [28. 22.] /e 0.084 /pscore 0 /loss 1.0843494 / Q_MAX -1.378279e+00
ts 473575 action_freq [16. 34.] /e 0.084 /pscore 0 /loss 0.22909354 / Q_MAX -7.140990e-01
ts 473625 action_freq [28. 22.] /e 0.084 /pscore 0 /loss 0.15530607 / Q_MAX -1.140434e+00
ts 473675 action_freq [36. 14.] /e 0.084 /pscore 0 /loss 0.0566257 / Q_MAX -9.860064e-01
ts 473725 action_freq [35. 15.] /e 0.084 /pscore 0 /loss 0.31277496 / Q_MAX -9.301603e-01
ts 473778 action_freq [48.  5.] /e 0.084 /pscore 0 /loss 0.43622607 / Q_MAX -1.089296e+00
ts 473828 act

In [ ]:
Jai Gurudev